In [40]:
#!/usr/bin/env python2

import pickle
import sys
import matplotlib.pyplot as plt
import numpy as np
import os

import denali
import subprocess
# change to the directory of this file
#os.chdir(os.path.dirname(__file__))

#import nnsample

PARAMS_FILE     = "all_params.npy"
TOPOLOGY_FILE   = "topology.pickle"
TARGETS_FILE    = "targets.npy"


def plot_net(net, targets):
    """Plots the neural network's predictions for sine on [0,2*pi]"""
    x = np.linspace(0, 2*np.pi, 100)

    y_sim = net.sim(x.reshape(-1,1))
    y_true = np.sin(x)

    plt.plot(x,y_sim)

    plt.plot(x,y_true, color='black', linestyle='--')

    plt.scatter(*targets.T, alpha=0.5)
    plt.show()


def main():

    # load the data
    parameters = np.load(PARAMS_FILE)
    
    #print parameters
    parameters = parameters.tolist()
    edges = denali.contour.kneighbors_complex(parameters, 10)
    
    denali.io.write_edges(open('edges', 'w'), edges)
    #denali.io.write_vertices(open('vertices', 'w'), )
    print "Computing contour tree..."
    subprocess.call(["ctree", "vertices", "edges", "tree.tree"])
    '''
    with open(TOPOLOGY_FILE) as f:
        topology = pickle.load(f)

    targets = np.load(TARGETS_FILE)

    #selection = denali.io.read_selection_file(sys.argv[1])

    # get the id of the child of the component
    #child = int(selection['component'][1][0])
    child = 100
    selected_param = parameters[child]
    print len(parameters)
    print len(selected_param)
    
    net = unpack_network(selected_param, topology)

    # Process(target=plot_net, args=[net, targets]).start()
    #plot_net(net, targets)
    #print topology
    #print topology['inp_minmax']
    #print topology['cn']
    '''
main()


/Users/wuzhenyu/anaconda/lib/python2.7/site-packages/sklearn/neighbors/graph.py:37: DeprecationWarning: The behavior of 'kneighbors_graph' when mode='connectivity' will change in version 0.18. Presently, the nearest neighbor of each sample is the sample itself. Beginning in version 0.18, the default behavior will be to exclude each sample from being its own nearest neighbor. To maintain the current behavior, set include_self=True.
  "behavior, set include_self=True.", DeprecationWarning)


Computing contour tree...


In [29]:
import neurolab as nl
import numpy as np

def layer_starting_indices(net, ci, cn):
    # the total number of parameters in each layer of the model
    n_params_per_layer = cn + (cn * ci)
    
    # the position of the beginning of the i-th layer in the param
    # vector. The last entry is the total number of params
    layer_starts = np.zeros(1 + len(net.layers), int)
    layer_starts[1:] = np.cumsum(n_params_per_layer)
    
    return layer_starts


def pack_network(net):
    """Packs the parameters of the feedforward net into an array.
    
    Returns:
        params - a numpy array of parameters
        topology - a description of the network's topology needed to construct
                   a network of the same size
    """
    cn = np.array([layer.cn for layer in net.layers])
    ci = np.array([layer.ci for layer in net.layers])
    
    # compute the starting indices of each layer
    layer_starts = layer_starting_indices(net, ci, cn)
    
    params = np.empty(layer_starts[-1])
    
    # flatten each layer's parameters
    for i,layer in enumerate(net.layers):
        flat_layer = np.concatenate((layer.np['b'],
                                     layer.np['w'].flatten()))
        begin = layer_starts[i]
        end = layer_starts[i+1]
        
        params[begin:end] = flat_layer
        
    # make a dictionary of the network's size and input information
    # this is necessary in order to construct a new network
    topology = dict(inp_minmax=net.inp_minmax,
                    cn=cn)
        
    return params, topology


def unpack_network(parameters, topology):
    """Builds a network from a packed parameter array."""
    # make a network with the same topology
    net = nl.net.newff(topology['inp_minmax'], topology['cn'])
    print 'Parameters:'
    print parameters
    print topology['inp_minmax']
    print topology['cn']
    print '**************************************************'
    cn = np.array([layer.cn for layer in net.layers])
    ci = np.array([layer.ci for layer in net.layers])
    print cn
    print ci
    print '**************************************************'
    # the total number of parameters in each layer of the model
    layer_starts = layer_starting_indices(net, ci, cn)

    for i,layer in enumerate(net.layers):
        begin = layer_starts[i]
        end = layer_starts[i+1]
        
        layer_params = parameters[begin:end]
        
        layer.np['b'] = layer_params[:layer.cn]
        layer.np['w'] = layer_params[layer.cn:].reshape((-1,layer.ci))
        print layer.np['b']
        print layer.np['w']
        print i
        print '++++++++++++++++++++++++++++++++++++++++++++++++++++++++++'
        
    return net